In [38]:
import os
import glob
import pandas as pd
from collections import Counter

In [107]:
path_corpus = '../../OE_data/texts/'
calgary_files = glob.glob(path_corpus + "calgary/*.txt")
calgary_all =  "out_calgary.txt"
# calgary_files[0]

In [108]:
def write_all(from_files, to_file):
    if os.path.exists(to_file):
        os.remove(to_file)
    with open(to_file, "wb") as out_file:
        for f in from_files:
            with open(f, "rb") as infile:
                out_file.write(infile.read())
write_all(calgary_files, calgary_all)

In [60]:
def get_words(fname):
    with open(fname, 'r') as f:
        return f.read().split()

In [63]:
all_words = get_words(calgary_all)
d = dict(Counter((w[0] for w in all_words)))
d['p']

132

In [103]:
df = pd.DataFrame(d.items(), columns=['letter', 'quantity'])
df = df.sort_values(by=['quantity'], ascending=False)
df = df.loc[~df['letter'].isin(['#', '.', ';', '!', ':', '(', ')', '?', '\'', '\"', ','])]
df.reset_index(drop=True, inplace=True)
df

,letter,quantity
0,h,19035
1,þ,17402
2,s,16850
3,w,14554
4,g,14541
5,o,11040
6,m,10818
7,f,9664
8,a,7742
9,e,7607


In [104]:
df.sum()

letter      hþswgomfaebnlidætcðurypkxqvzᛟ
quantity                           175394
dtype: object

In [105]:
len(list(all_words))

241108